# Assignment

In this assignment we will train a multiple (multivariable) linear regression model, extending the single variable linear regression model in the tutorial. 

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

Use the following lines to import any needed libraries:

In [2]:
import os, numpy as np
from tensorflow import losses, optimizers, keras
from tensorflow.keras import Input, Model, models, layers

# Data

As in the tutorial, data for this assignment will be simulated via a Python generator:

In [3]:
def Generator(batch_size=128):
    """
    Method to define a Python generator for training data
    
    """
    # --- Define lambda function for random values [-0.5, +0.5]
    lo = -0.5
    hi = +0.5
    rand = lambda shape : np.random.rand(*shape) * (hi - lo) + lo
    
    # --- Define lambda function for linear transform
    m = np.array([0.2, 0.8, -0.3, 0.4, -0.5]).reshape(5, 1)
    b = 0.7
    f = lambda x : np.matmul(x, m) + b + rand((x.shape[0], 1))
    
    while True:
        
        x = np.random.rand(batch_size, 5)
        y = f(x)
        
        yield {'x': x, 'y': y}

**Imporant**: do *not* change the code for the above Generator to ensure that your results are consistent.

# Training

In this assignment we will train a multiple (multivariable) linear regression model, extending the single variable linear regression model in the tutorial The provided input to the model will comprise of a five-element vector, and the output will be a single-element numeric prediction.  

### Define the backbone model

In [4]:
gen_train = Generator()

In [5]:
# --- Define backbone model
x = Input(shape = (5,))
op = layers.Dense(1)
y = op(x)
backbone = Model(inputs=x, outputs=y)

### Define the training model 

In [6]:
# --- Define inputs and y_pred
inputs = {
    'x': Input(shape=(5,), name = 'x'),
    'y': Input(shape= (1,), name = 'y')
}
y_pred = backbone(inputs['x'])

In [7]:
# --- Define loss
loss = ((inputs['y']-y_pred) **2)

In [8]:
# --- Define training model
training = Model(inputs=inputs, outputs={'y_pred': y_pred, 'loss': loss})
training.add_loss(loss)

In [9]:
training.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 x (InputLayer)                 [(None, 5)]          0           []                               
                                                                                                  
 y (InputLayer)                 [(None, 1)]          0           []                               
                                                                                                  
 model (Functional)             (None, 1)            6           ['x[0][0]']                      
                                                                                                  
 tf.math.subtract (TFOpLambda)  (None, 1)            0           ['y[0][0]',                      
                                                                  'model[0][0]']            

### Compile the model

In [10]:
# --- Define loss and optimizer
optimizer = optimizers.Adam(learning_rate=1e-3)

# --- Compile model
training.compile(optimizer=optimizer)

### Train the model

In [11]:
training.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=20)

Epoch 1/20
500/500 [==============================] - 2s 2ms/step - loss: 0.6572
Epoch 2/20
500/500 [==============================] - 1s 2ms/step - loss: 0.1761
Epoch 3/20
500/500 [==============================] - 1s 2ms/step - loss: 0.1458
Epoch 4/20
500/500 [==============================] - 1s 2ms/step - loss: 0.1229
Epoch 5/20
500/500 [==============================] - 1s 2ms/step - loss: 0.1049
Epoch 6/20
500/500 [==============================] - 1s 2ms/step - loss: 0.0933
Epoch 7/20
500/500 [==============================] - 1s 2ms/step - loss: 0.0873
Epoch 8/20
500/500 [==============================] - 1s 2ms/step - loss: 0.0855
Epoch 9/20
500/500 [==============================] - 1s 2ms/step - loss: 0.0838
Epoch 10/20
500/500 [==============================] - 1s 2ms/step - loss: 0.0834
Epoch 11/20
500/500 [==============================] - 1s 2ms/step - loss: 0.0833
Epoch 12/20
500/500 [==============================] - 1s 2ms/step - loss: 0.0835
Epoch 13/20
500/500 [====

# Evaluation

Based on the tutorial discussion, use the following cells to check your algorithm performance. Consider loading a saved model and running prediction using `training.predict(...)` on a batch (or several batches) of data. 

In [12]:
data = next(gen_train)

In [13]:
data['x'].shape

(128, 5)

In [14]:
np.mean((training.predict(data)['y_pred'] - data['y']) **2)

0.08021622769596364

In [15]:
data['x'].shape

(128, 5)

In [16]:
np.mean(training.predict(data)['loss'])

0.08021623

# Submission

Use the following line to save your `backbone` model for submission:

In [17]:
# --- Serialize a model
fname = './wjhan_model.hdf5'
backbone.save(fname)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooked file would be submitted under the name `panteater_notebook.ipynb` and his model file would be submitted under the name `panteater_model.hdf5`.